In [1]:
!pip install langchain openai wikipedia langchain-community "httpx==0.27.2"

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.3 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=017efc0a0b332a36de9d679dd1cea7b2c4aebc941e5dd4931c96fb5664fc8d75
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
import os
from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = "sk" # sk-로 시작하는 API 키를 입력하세요

llm = ChatOpenAI(model="gpt-4o-2024-11-20")

<ipython-input-2-dd90825fc03e>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-2024-11-20")


In [3]:
#제로샷 ReAct 에이전트
from langchain.agents import initialize_agent, load_tools, AgentType
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [4]:
agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True) # 대화형 에이전트를 초기화
agent.invoke("소나무 옮겨심기 좋은 계절은?") # 에이전트에게 질문을 전달하고 답변을 생성하도록 실행

<ipython-input-4-e57e329b280e>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True) # 대화형 에이전트를 초기화




> Entering new AgentExecutor chain...
소나무 옮겨심기 좋은 계절을 알기 위해 일반적인 조경 및 나무 심기와 관련된 정보를 찾는 것이 필요하다.  
Action: wikipedia  
Action Input: 소나무 옮겨심기 좋은 계절  

Observation: No good Wikipedia Search Result was found
Thought:소나무와 같은 나무를 옮겨 심는 데 적합한 계절은 일반적으로 나무의 생장 속도가 느려지고 물 수요가 줄어드는 시기인 가을이나 이른 봄이다. 이 시기에는 뿌리가 안정적으로 자리를 잡을 수 있는 조건을 제공하기 쉽다. 그러나 더 구체적인 답변이 필요하면 조경 전문가의 조언을 구하는 것이 좋다.

Final Answer: 소나무 옮겨심기에 적합한 계절은 일반적으로 가을이나 이른 봄이다.

> Finished chain.


{'input': '소나무 옮겨심기 좋은 계절은?', 'output': '소나무 옮겨심기에 적합한 계절은 일반적으로 가을이나 이른 봄이다.'}

In [5]:
#Conversational ReAct
from langchain.memory import ConversationBufferMemory
tools = load_tools(["llm-math"],  llm=llm) # 에이전트가 사용할 도구 지정, 수학 관련 문제를 해결하는 도구
memory = ConversationBufferMemory(memory_key="chat_history") # 대화 기록을 저장할 키를 지정
conversational_agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True, # 디버깅 목적으로 에이전트의 내부 동작을 출력
    max_iterations=3, # 에이전트가 질문을 해결하기 위해 도구를 사용할 수 있는 최대 반복 횟수를 3으로 지
    memory=memory,)
conversational_agent.invoke("에드시런은 누구이며 2025년 현재 나이는 몇살이야?")

<ipython-input-5-af403171cba8>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history") # 대화 기록을 저장할 키를 지정




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: Calculator  
Action Input: 2025 - 1991  

Observation: Answer: 34
Thought:Do I need to use a tool? No  
AI: 에드 시런(Ed Sheeran)은 영국의 가수, 작곡가, 그리고 음악 프로듀서입니다. 그는 "Shape of You", "Perfect", "Thinking Out Loud"와 같은 히트곡들로 잘 알려져 있으며, 전 세계적으로 많은 사랑을 받고 있는 아티스트입니다. 에드 시런은 1991년 2월 17일생으로, 2025년에는 만 34세입니다.

> Finished chain.


{'input': '에드시런은 누구이며 2025년 현재 나이는 몇살이야?',
 'chat_history': '',
 'output': '에드 시런(Ed Sheeran)은 영국의 가수, 작곡가, 그리고 음악 프로듀서입니다. 그는 "Shape of You", "Perfect", "Thinking Out Loud"와 같은 히트곡들로 잘 알려져 있으며, 전 세계적으로 많은 사랑을 받고 있는 아티스트입니다. 에드 시런은 1991년 2월 17일생으로, 2025년에는 만 34세입니다.'}

In [6]:
#Self Ask With Search Agent 사용
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [7]:
import os
os.environ["TAVILY_API_KEY"] = "tvly" # tvly로 시작하는 API 키를 입력하세요

In [8]:
tools = [TavilyAnswer(max_results=5, name="Intermediate Answer")] # 도구 이름을 설정하여 나중에 에이전트가 이 도구를 참조할 때 사용
prompt = hub.pull("hwchase17/self-ask-with-search") # 랭체인에서 제공하는 프롬프트 허브에서 self-ask-with-search 프롬프트를 가져옵니

agent = create_self_ask_with_search_agent(llm, tools, prompt) # "Self Ask With Search" 기능을 구현하는 에이전트를 생성

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True) #AgentExecutor 이용하여 에이전트 사용
agent_executor.invoke({"input": "AI Agent를 세 문장으로 설명해줘"})

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'input': 'AI Agent를 세 문장으로 설명해줘',
 'output': 'AI Agent는 인공지능 기술을 활용하여 특정 작업이나 문제를 해결하도록 설계된 소프트웨어 프로그램입니다. 주어진 데이터를 분석하고 학습하여 의사결정을 내리거나 행동을 수행할 수 있습니다. 다양한 분야에서 자동화와 효율성을 높이는 데 사용됩니다.'}

In [9]:
# ReAct document store
from langchain.docstore.wikipedia import Wikipedia # 문서 저장소로 위키피디아 이용
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer # 문서가 저장된 데이터베이스(위키피디아)를 탐색하는 도구

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,  # 용어에 대한 키워드 기반의 검색, 사용자가 정확히 무엇을 찾고자 하는지 모를 때 유용
    description="docstore에서 용어를 검색하세요.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup, # 고유하게 식별할 수 있는 키를 이용한 검색, 특정 문서(또는 항목)를 정확하게 찾을 때 사용
    description="docstore에서 용어 검색.",
  )
] # 에이전트가 사용할 수 있는 도구를 정의

<ipython-input-9-9c1be38f05f3>:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  docstore=DocstoreExplorer(Wikipedia())


In [10]:
react = initialize_agent(tools = tools,
                         llm = llm,
                         agent = AgentType.REACT_DOCSTORE, # 에이전트 유형을 ReAct document store로 지정
                         handle_parsing_errors=True, # 에이전트가 작업을 수행하는 동안 발생하는 구문 오류(Parsing Errors)를 처리
                         max_iterations=1, # 에이전트가 최대 몇 번까지 작업을 반복할지 지정
                         max_execution_time=1, # 에이전트가 작업을 수행하는 최대 시간을 초 단위로 설정
                         verbose = True,) # 랭체인 에이전트 초기화

In [11]:
def query_data(query):
    try:
        response = react.invoke(query)  # 에이전트에게 쿼리 전달
        print("----------------------", response)  # 결과 출력(디버깅용)
        return response  # 검색 결과 반환
    except Exception as e:
        print(f"Error: {e}")  # 에러 메시지를 출력
        raise  # 에러 다시 발생(추가 디버깅 용도)

query = "밍크 선인장 키우는 방법은?"
response = query_data(query)
print(response['output']) # 에이전트의 최종 답변 출력



> Entering new AgentExecutor chain...
Could not parse LLM Output: 밍크 선인장은 선인장의 한 종류로, 키우는 방법은 다음과 같습니다:

1. **햇빛**: 밍크 선인장은 햇빛을 좋아하므로 밝고 직사광선이 드는 곳에 두는 것이 좋습니다. 하지만 너무 강한 직사광선은 피하고, 반그늘에서 키울 수도 있습니다.

2. **물 주기**: 밍크 선인장은 과습에 약하기 때문에 물을 자주 주지 않는 것이 중요합니다. 흙이 완전히 마른 후에 물을 주며, 겨울철에는 물 주는 횟수를 줄입니다.

3. **흙**: 배수가 잘 되는 선인장 전용 흙이나 모래가 섞인 흙을 사용하는 것이 좋습니다.

4. **온도**: 따뜻한 온도를 선호하며, 추위에 약합니다. 겨울철에는 10°C 이상의 온도를 유지하는 것이 좋습니다.

5. **비료**: 성장기(봄, 여름)에는 한 달에 한 번 정도 선인장용 비료를 소량 주는 것이 좋습니다.

6. **화분**: 밑에 배수 구멍이 있는 화분을 사용하는 것이 중요하며, 화분의 크기는 식물의 크기에 맞게 선택합니다.

7. **환기**: 공기가 잘 통하는 곳에서 키우는 것이 좋습니다. 습기가 오래 머무르는 환경은 피해야 합니다.

위의 방법을 따라 꾸준히 관리하면 밍크 선인장을 건강하게 키울 수 있습니다!
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

> Finished chain.
---------------------- {'input': '밍크 선인장 키우는 방법은?', 'output': 'Agent stopped due to iteration limit or time limit.'}
Agent stopped due to iteration limit or t